In [1]:
import numpy as np
import pandas as pd
import itertools

from scipy import stats
from statsmodels.stats.descriptivestats import sign_test
from statsmodels.stats.weightstats import zconfint

In [2]:
import scipy
import statsmodels

In [10]:
sample4 = [49,58,75,110,112,132,151,276,281,362]
for i in np.arange(len(sample4)):
    sample4[i] -= 200

In [12]:
stats.wilcoxon(sample4)

WilcoxonResult(statistic=17.0, pvalue=0.2845026979112075)

In [17]:
sample5_1 = [22, 22, 15, 13, 19, 19, 18, 20, 21, 13, 13, 15,22,22,15,13,19,19,18,20,21,13,13,15]
sample5_2 = [17, 18, 18, 15, 12, 4, 14, 15, 10.17,18,18,15,12,4,14,15,10]

In [64]:
stats.mannwhitneyu(sample5_1, sample5_2, alternative='greater')

MannwhitneyuResult(statistic=310.0, pvalue=0.0024476071679050988)

In [22]:
challenger = pd.read_csv('challenger.txt', sep='\t')
challenger.describe()
challenger.info()
challenger.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23 entries, 0 to 22
Data columns (total 3 columns):
Unnamed: 0     23 non-null object
Temperature    23 non-null float64
Incident       23 non-null int64
dtypes: float64(1), int64(1), object(1)
memory usage: 632.0+ bytes


,Unnamed: 0,Temperature,Incident
0,Apr12.81,18.9,0
1,Nov12.81,21.1,1
2,Mar22.82,20.6,0
3,Nov11.82,20.0,0
4,Apr04.83,19.4,0


In [29]:
temp1 = challenger[(challenger['Incident'] == 1)]['Temperature'].to_numpy()

In [31]:
temp0 = challenger[(challenger['Incident'] == 0)]['Temperature'].to_numpy()

In [34]:
def get_bootstrap_samples(data, n_samples):
    indices = np.random.randint(0, len(data), (n_samples, len(data)))
    samples = data[indices]
    return samples

In [35]:
n_samples = 1000
np.random.seed(seed=0)
sample1 = get_bootstrap_samples(temp1, n_samples)
sample0 = get_bootstrap_samples(temp0, n_samples)

In [49]:
sas1 = np.mean(sample1, axis = 1)
sas0 = np.mean(sample0, axis = 1)

In [51]:
sas1.shape

(1000,)

In [36]:
def stat_intervals(stat, alpha):
    boundaries = np.percentile(stat, [100 * alpha / 2., 100 * (1 - alpha / 2.)])
    return boundaries

In [50]:
stat_intervals(sas1 - sas0, 0.05)

array([-8.06457589, -1.45040179])

In [57]:
def permutation_t_stat_ind(sample1, sample2):
    return np.mean(sample1) - np.mean(sample2)

In [52]:
def permutation_t_stat_1sample(sample, mean):
    t_stat = sum(list(map(lambda x: x - mean, sample)))
    return t_stat

In [53]:
def get_random_combinations(n1, n2, max_combinations):
    index = list(range(n1 + n2))
    indices = set([tuple(index)])
    for i in range(max_combinations - 1):
        np.random.shuffle(index)
        indices.add(tuple(index))
    return [(index[:n1], index[n1:]) for index in indices]

In [54]:
def permutation_zero_dist_ind(sample1, sample2, max_combinations = None):
    joined_sample = np.hstack((sample1, sample2))
    n1 = len(sample1)
    n = len(joined_sample)
    
    if max_combinations:
        indices = get_random_combinations(n1, len(sample2), max_combinations)
    else:
        indices = [(list(index), list(filter(lambda i: i not in index, range(n)))) \
                    for index in itertools.combinations(range(n), n1)]
    
    distr = [joined_sample[list(i[0])].mean() - joined_sample[list(i[1])].mean() \
             for i in indices]
    return distr

In [55]:
def permutation_test(sample, mean, max_permutations = None, alternative = 'two-sided'):
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError("alternative not recognized\n"
                         "should be 'two-sided', 'less' or 'greater'")
    
    t_stat = permutation_t_stat_ind(sample, mean)
    
    zero_distr = permutation_zero_dist_ind(sample, mean, max_permutations)
    
    if alternative == 'two-sided':
        return sum([1. if abs(x) >= abs(t_stat) else 0. for x in zero_distr]) / len(zero_distr)
    
    if alternative == 'less':
        return sum([1. if x <= t_stat else 0. for x in zero_distr]) / len(zero_distr)

    if alternative == 'greater':
        return sum([1. if x >= t_stat else 0. for x in zero_distr]) / len(zero_distr)

In [61]:
np.random.seed(0)
print("p-value: %f" % permutation_test(temp1, temp0, max_permutations = 10000, alternative = 'two-sided'))

p-value: 0.005700
